# Run Model 1
Dit notebook kan gebruikt worden om BrfRFE_model1 te draaien. Dit model is bedoeld om te voorspellen of een kandidaat starter zal worden op basis van de data die verzamelt is voordat het eerste telefonisch contact is geweest. 

## Install packages

In [ ]:
##

## Import Libraries

In [ ]:
from datetime import datetime
import pandas as pd

## Upload data
Je kan de data van de kandidaten waarover je wilt voorspellen of ze starter worden uploaden in de map "kandidaat_data". Dit dient in de vorm van een excel te zijn met de naam "kandidaten_[DATUM].xlsx". Op de plek van [DATUM] vul je de datum in van uploaden in het format yyyymmdd, zonder streepjes of iets er tussen. Bijvoorbeeld een data bestand geupload op 11 augustus 2024 heeft de naam "kandidaten_20240811.xlsx". 

De excel heeft een aantal vaste kolommen en kolomnamen. Een voorbeeld excel bevind zich in kandidaten_data/voorbeeld_kandidaten.xlsx. Let op dat wijzigingen in de namen van kolommen of antwoordcategorieen impact hebben op de bruikbaarheid van de data en de kwaliteit van het model. 

## Lees data in 

In [ ]:
df_kandidaten = pd.read_excel(f"kandidaat_data/kandidaten_{str(datetime.today().date()).replace("-","")}.xlsx")

## Lees model in